Let's bootstrap!  Recall that bootstrapping is sampling with replacement from a dataset to produce a new dataset. Bootstrapping is used in random forests to guard against overfitting. It also has wide application in many other areas of statistics - let's see two of them.

1) Produce a bootstrapped estimate of the median and 95 percent confidence interval over the median of the dependent variable in the [attached dataset](pair_boot.csv).

2) Use the attached data to run the linear model y = xb. Produce bootstrapped estimates of the model parameters, b, and a 95% confidence interval over them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# makes data
def maker(N,n_vars):
    """A function to generate Monte Carlo linear regression data"""
    x = []    #an empty list to hold the data
    y = np.zeros(N)    #an array to hold the dependent variable
    b = []    #an empty list to hold the true bs
    i = 1
    while i <= n_vars:    #loop over the variables we want to create
        x_i = np.random.normal(loc = 0.0, scale = 1.0, size = N)    #generate the data
        x.append(x_i)    #add it to the list of data
        b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random effect for this variable
        b.append(b_i)    #add it to the list of effects
        y = y + b_i*x_i    #add the variable effect to the dependent variable
        i += 1    #index up i
    
    x.append(np.ones(N))    #and a column of ones for a constant
    b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random intercept
    b.append(b_i)    #append this intercept to the effects
    y = b_i + y + np.random.normal(loc = 0.0, scale = 1.0, size = N)    #add the normally distributed error term and the intercept
    return [np.array(x).T,np.array(y),np.array(b)]

In [ ]:
def bootstrapped_median_CI(y,n_boot):
    """A function to produce a bootstrapped 95% CI over the median of a dataset.
    Takes in:
    y: an array or list of data from which you want to compute the median
    n_boot: the number of bootstrap iterations
    """
    medians = []    #an empty list to hold the medians
    for _ in range(n_boot):    #loop over the number of bootstraps
        medians.append(     np.median(   y[np.random.randint(0,len(y),len(y)-1)]   ))    #sample the data and compute the median
    return [np.mean(medians),np.percentile(medians,97.5),np.percentile(medians,2.5)]#,medians]

In [ ]:
def bootstrapped_regression(X,y,n_boot):
    """A function to produce a bootstrapped 95% CI over regression coefficients.
    Takes in:
    X: an aray of data on the dependent variable
    y: an array or list of data for the dependent variable
    n_boot: the number of bootstrap iterations
    """
    coefficients = []    #an empty list to hold the coefficients
    for _ in range(n_boot):    #loop over the number of bootstraps
        boot = np.random.randint(0,len(y)-1,len(y))    #sample the data
        y_boot = y[boot]    #take the sampled ys
        X_boot = X[boot]    #take the sampled xs
        coefficients.append(np.linalg.inv(X_boot.T.dot(X_boot)).dot(X_boot.T.dot(y_boot)))    #compute the regression coefficients
    return [np.mean(np.array(coefficients).T,1),np.percentile(np.array(coefficients).T,97.5,1),np.percentile(np.array(coefficients).T,2.5,1)]

In [ ]:
N = 1000    #number of observations
n_vars = 5    #number of variables
Data = maker(N,n_vars)    #call the function to make the data
X = Data[0]
y = Data[1]

In [ ]:
bootstrapped_median_CI(y=y,n_boot=100)

In [ ]:
bootstrapped_regression(X=X,y=y,n_boot=100)

In [ ]:
# true median
np.median(y)

In [ ]:
# true coefficients
np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))